In [5]:
import wbgapi as wb
import pandas as pd
import sqlite3

# Indicators
indicators = {
    "School enrollment, primary (% gross)": "SE.PRM.ENRR",
    "School enrollment, secondary (% gross)": "SE.SEC.ENRR",
    "School enrollment, tertiary (% gross)": "SE.TER.ENRR",
}

# Country codes and names
countries = {
    "JPN": "Japan",
    "AFG": "Afghanistan",
    "IND": "India"
}

# Create SQLite in-memory database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Prepare and insert data
all_data = []

for code, name in countries.items():
    data = wb.data.DataFrame(indicators.values(), code, mrv=200)
    if not data.empty:
        df = data.transpose()
        df.reset_index(inplace=True)
        df["Country"] = name
        df = df.rename(columns={"index": "Year"})
        all_data.append(df)

# Combine all country data into one DataFrame
combined_df = pd.concat(all_data)

# Write to SQL database
combined_df.to_sql("enrollment_data", conn, index=False, if_exists="replace")

139

In [6]:
split_queries = [
    """
    CREATE TABLE afghanistan AS
    SELECT * FROM enrollment_data
    WHERE Country = 'Afghanistan'
    """,
    """
    CREATE TABLE india AS
    SELECT * FROM enrollment_data
    WHERE Country = 'India'
    """,
    """
    CREATE TABLE japan AS
    SELECT * FROM enrollment_data
    WHERE Country = 'Japan'
    """
]

for q in split_queries:
    cursor.execute(q)



In [7]:
conn.execute('ALTER TABLE india RENAME COLUMN "SE.PRM.ENRR" TO "Primary Enrollment";')
conn.execute('ALTER TABLE india RENAME COLUMN "SE.SEC.ENRR" TO "Secondary Enrollment";')
conn.execute('ALTER TABLE india RENAME COLUMN "SE.TER.ENRR" TO "Tertiary Enrollment";')

conn.execute('ALTER TABLE japan RENAME COLUMN "SE.PRM.ENRR" TO "Primary Enrollment";')
conn.execute('ALTER TABLE japan RENAME COLUMN "SE.SEC.ENRR" TO "Secondary Enrollment";')
conn.execute('ALTER TABLE japan RENAME COLUMN "SE.TER.ENRR" TO "Tertiary Enrollment";')

conn.execute('ALTER TABLE afghanistan RENAME COLUMN "SE.PRM.ENRR" TO "Primary Enrollment";')
conn.execute('ALTER TABLE afghanistan RENAME COLUMN "SE.SEC.ENRR" TO "Secondary Enrollment";')
conn.execute('ALTER TABLE afghanistan RENAME COLUMN "SE.TER.ENRR" TO "Tertiary Enrollment";')

In [5]:
pd.read_sql_query("SELECT * FROM india", conn).to_csv("india_enrollment.csv", index=False)
pd.read_sql_query("SELECT * FROM japan", conn).to_csv("japan_enrollment.csv", index=False)
pd.read_sql_query("SELECT * FROM afghanistan", conn).to_csv("afghanistan_enrollment.csv", index=False)

**Descriptive Statistics**

In [4]:
def get_descriptive_stats(table_name):
    query = f"""
    SELECT 
      COUNT(*) AS count,
      AVG("Primary Enrollment") AS avg_primary,
      MIN("Primary Enrollment") AS min_primary,
      MAX("Primary Enrollment") AS max_primary,
      AVG("Secondary Enrollment") AS avg_secondary,
      MIN("Secondary Enrollment") AS min_secondary,
      MAX("Secondary Enrollment") AS max_secondary,
      AVG("Tertiary Enrollment") AS avg_tertiary,
      MIN("Tertiary Enrollment") AS min_tertiary,
      MAX("Tertiary Enrollment") AS max_tertiary
    FROM {table_name};
    """
    return pd.read_sql_query(query, conn)

# Descriptive statistics for each table
india_stats = get_descriptive_stats("india")
japan_stats = get_descriptive_stats("japan")
afghanistan_stats = get_descriptive_stats("afghanistan")

# Print the results
print("India:\n", india_stats)
print("\nJapan:\n", japan_stats)
print("\nAfghanistan:\n", afghanistan_stats)

India:
    count  avg_primary  min_primary  max_primary  avg_secondary  min_secondary  \
0     53    96.764678    78.981659   119.512833      51.181328       24.17012   

   max_secondary  avg_tertiary  min_tertiary  max_tertiary  
0      81.177861     13.906582       4.92445     33.116524  

Japan:
    count  avg_primary  min_primary  max_primary  avg_secondary  min_secondary  \
0     39   102.726731    98.300819   106.348808      97.819013      89.496887   

   max_secondary  avg_tertiary  min_tertiary  max_tertiary  
0     105.892799     38.852622      16.85821      64.62883  

Afghanistan:
    count  avg_primary  min_primary  max_primary  avg_secondary  min_secondary  \
0     47    62.809384     21.81455   110.007507      26.310449        8.88566   

   max_secondary  avg_tertiary  min_tertiary  max_tertiary  
0      57.039299      2.935639       0.80651      10.85844  
